# Explorando o espaço dos compostos químicos com um sistema de recomendação gráfica

Neste caderno demonstramos várias maneiras pelas quais nosso sistema de recomendação de grafos (GR) pode ser usado para explorar o espaço de compostos químicos. Primeiro exemplificamos (1) como obter recomendações ao longo da hiperdireção composicional, demonstrando como recomendar novas ocupações de sítios atômicos. Em seguida, mostramos (2) como o GR pode produzir recomendações ao longo da hiperdireção estrutural, recomendando novas estruturas cristalinas para uma composição química específica. Finalmente, mostramos (3) como obter recomendações ao longo de hiperdireções híbridas (composicional-estruturais): demonstramos como obter recomendações de locais para um determinado átomo ocupar; também mostramos como extrair recomendações de novos compostos e estruturas em um determinado sistema químico.

## 1. Recomendações ao longo da hiperdireção composicional

### 1.1 Demonstrando recomendações de ocupação de locais atômicos

Esta seção orienta você através do processo de obtenção de novas ocupações de sítios atômicos com base em um determinado ID OQMD que representa um protótipo estrutural de interesse ou em um arquivo de estrutura cristalina.

Como exemplo, focaremos no protótipo de estrutura cristalina de [$\text{CsV}_3\text{Sb}_5$](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.125.247002) , um composto que recentemente despertou interesse por suas novas propriedades. A visualização desta estrutura cristalina única é mostrada abaixo:

<p align="center">
<img src="etc/kagome_structure.png" alt="Crystal structure of CsK3Sb5" width="70%" height="auto" align="center">
</p>

Correspondente a esta estrutura, a entrada do OQMD com o ID: 1514955 pode ser encontrada neste [link](https://oqmd.org/materials/entry/1514955). Nosso objetivo neste caderno é demonstrar o processo de recomendação de novas ocupações de locais iônicos usando este protótipo estrutural como referência.

In [1]:
# If you are using Colab, remove the comments below and run this cell. 

# !pip install gitpython

#import git
#import os

#git.Repo.clone_from('https://github.com/simcomat/ionic-sub-RS', '/content/ionic-sub-RS')
#os.chdir('/content/ionic-sub-RS')
# !pip install -r requirements.txt

In [ ]:
## Conferir instalação de versões compativeis com modelo pre-treinado
# !pip install "scipy<1.13"
# !pip install --upgrade gensim
# !pip install scikit-learn==1.2.1 joblib

# !pip show scikit-learn
# !pip show numpy
# !pip show joblib

As classes usadas para construir o Sistema de Recomendação são definidas no arquivo `recommender/core.py`.

In [5]:
from recommender.core import RecommenderSystem, OccupationData
import joblib
from gensim.models import Word2Vec
from pymatgen.core import Element, Structure
from pandas import DataFrame

#### Carregando a incorporação e definindo o local dos dados

Nesta seção, estabelecemos o caminho para os dados de ocupação que foram usados para construir o gráfico NetworkX. Este gráfico pode ser encontrado em `data/G100.gexf`. Simultaneamente, carregamos o modelo Word2Vec, que inclui os embeddings de íons e sites.

Também carregamos o classificador Decision Tree para determinar o limite de distância aplicado no espaço de incorporação.

Aqui estamos definindo íons que não desejamos que sejam recomendados (gases nobres e elementos radioativos)

In [6]:
clf = joblib.load(f'models/decision_tree_100.joblib')
clf

DecisionTreeClassifier(max_depth=1)

In [7]:
json_file_path = 'data/occupancy_data.json.gz'

model = Word2Vec.load(f'models/word2vec_100.model')
embedding = model.wv

clf = joblib.load(f'models/decision_tree_100.joblib')
distance_threshold = clf.tree_.threshold[0]

In [8]:
ions = ['Xe', 'Kr', 'Ar', 'Ne', 'He', 'I', 'Br', 'Cl', 'F', 'Te', 'Se', 'S', 'O', 'Bi', 'Sb', 'As', 'P', 'N', 'Pb', 'Sn', 'Ge', 'Si', 'C', 'Tl', 'In', 'Ga', 'Al', 'B', 'Hg', 'Cd', 'Zn', 'Au', 'Ag', 'Cu', 'Pt', 'Pd', 'Ni', 'Ir', 'Rh', 'Co', 'Os', 'Ru', 'Fe', 'Re', 'Tc', 'Mn', 'W', 'Mo', 'Cr', 'Ta', 'Nb', 'V', 'Hf', 'Zr', 'Ti', 'Pu', 'Np', 'U', 'Pa', 'Th', 'Ac', 'Lu', 'Yb', 'Tm', 'Er', 'Ho', 'Dy', 'Tb', 'Gd', 'Eu', 'Sm', 'Pm', 'Nd', 'Pr', 'Ce', 'La', 'Y', 'Sc', 'Ba', 'Sr', 'Ca', 'Mg', 'Be', 'Cs', 'Rb', 'K', 'Na', 'Li', 'H']
ion_forbidden_list = [ion for ion in ions if (Element(ion).Z > 83 or 
                                              ion in ['Tc', 'Pm'] or
                                              Element(ion).group == 18)]
ion_forbidden_list 

['Xe', 'Kr', 'Ar', 'Ne', 'He', 'Tc', 'Pu', 'Np', 'U', 'Pa', 'Th', 'Ac', 'Pm']

Primeiro, criamos um objeto `OccupationData` com o caminho do arquivo json compactado.

In [9]:
occupation_data = OccupationData(json_file_path)

Nesta fase, recuperamos um objeto `AnonymousMotif` para um ID OQMD específico. Este objeto encapsula todos os dados da estrutura cristalina, conforme definido pelo conceito Anonymous Motif descrito no artigo. Essas informações centradas na estrutura são a base para gerar nossas recomendações de ocupação de locais iônicos.

In [10]:
kagome_id = 1514955
kagome_AM = occupation_data.get_AM_from_OQMD_id(kagome_id)

O objeto `AnonymousMotif` fornece um protótipo de estrutura Pymatgen consistindo exclusivamente de átomos de carbono (C). Ele também vem com índices de sites equivalentes. Após receber as recomendações de ocupação, esses recursos podem ser utilizados para executar substituições iônicas, transformando efetivamente o protótipo em novas estruturas materiais.

In [11]:
kagome_AM.example_structure

Structure Summary
Lattice
    abc : 5.469073129 5.469073129363581 9.384302298
 angles : 90.0 90.0 120.00000000384938
 volume : 243.08607542522887
      A : 5.469073129 0.0 0.0
      B : -2.734536565 4.736356265 0.0
      C : 0.0 0.0 9.384302298
    pbc : True True True
PeriodicSite: C (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: C (2.735, 1.579, 2.411) [0.6667, 0.3333, 0.2569]
PeriodicSite: C (-3.333e-10, 3.158, 2.411) [0.3333, 0.6667, 0.2569]
PeriodicSite: C (0.0, 0.0, 4.692) [0.0, 0.0, 0.5]
PeriodicSite: C (2.735, 1.579, 6.974) [0.6667, 0.3333, 0.7431]
PeriodicSite: C (-3.333e-10, 3.158, 6.974) [0.3333, 0.6667, 0.7431]
PeriodicSite: C (2.735, 0.0, 4.692) [0.5, 0.0, 0.5]
PeriodicSite: C (-1.367, 2.368, 4.692) [0.0, 0.5, 0.5]
PeriodicSite: C (1.367, 2.368, 4.692) [0.5, 0.5, 0.5]

In [12]:
kagome_AM.equivalent_sites_indexes

[0, 1, 1, 3, 1, 1, 6, 6, 6]

A seguir, instanciaremos um objeto `RecommenderSystem`. Este processo envolve a integração de vários elementos, incluindo `OccupationData`, incorporações de nós, um limite de distância predefinido (fornecido pela árvore de decisão treinada) e uma lista de íons que são considerados inadequados para inclusão (`ion_forbidden_list`).

Pode-se refinar ainda mais as recomendações do modelo definindo um limite de distância menor, o que torna o processo de recomendação mais rigoroso.

In [13]:
rs = RecommenderSystem(occupation_data=occupation_data, 
                       embedding=embedding, 
                       distance_threshold=distance_threshold, 
                       ion_forbidden_list=ion_forbidden_list)

Para gerar recomendações para esta estrutura de protótipo, alimentamos o `AnonymousMotif` - derivado do ID OQMD - no método `get_recommendations_for_AM` do `RecommenderSystem`. A saída é um dicionário onde cada chave corresponde a um objeto `AMsite` representando um site específico do Anonymous Motif. Cada chave está associada a um valor que compreende uma lista de íons recomendados para aquele site específico.

As recomendações são expressas como tuplas no formato `(íon, distância íon-sítio, nova ocupação)`. Aqui, 'nova ocupação' indica se a ocupação sugerida do local iônico representa uma nova variante dentro do conjunto composto OQMD (o conjunto de dados usado para construir o sistema de recomendação). As recomendações são classificadas com base na distância íon-local.

É importante ressaltar que as chaves `AMSite` exibem seus respectivos índices de site. Esses índices fazem referência a locais específicos em `AnonymousMotif.example_structure` onde os íons recomendados devem ser substituídos, fornecendo assim um mapeamento claro para possíveis novos compostos.

In [14]:
kagome_recommendations = rs.get_recommendation_for_AM(kagome_AM)
kagome_recommendations

{'191_9_1(6/mmm)_1(6/mmm)_3(mmm)_4(3m)_0[0:1(6/mmm)]': [('Rb',
   0.018323421478271484,
   False),
  ('K', 0.021883368492126465, False),
  ('Cs', 0.02744007110595703, False),
  ('Tl', 0.1181100606918335, False),
  ('Na', 0.13025522232055664, False),
  ('Ba', 0.26111143827438354, True)],
 '191_9_1(6/mmm)_1(6/mmm)_3(mmm)_4(3m)_0[1:1(6/mmm)]': [('Sb',
   0.003526031970977783,
   False),
  ('Bi', 0.028452038764953613, False),
  ('As', 0.07530730962753296, False),
  ('P', 0.15748441219329834, False),
  ('Ir', 0.24137485027313232, True),
  ('Pt', 0.28948670625686646, True),
  ('Ta', 0.2916147708892822, True),
  ('Re', 0.30770790576934814, True),
  ('Rh', 0.3191390037536621, True),
  ('Au', 0.3234303593635559, True),
  ('Os', 0.3247630000114441, True)],
 '191_9_1(6/mmm)_1(6/mmm)_3(mmm)_4(3m)_0[2:3(mmm)]': [('V',
   0.03470265865325928,
   False),
  ('Nb', 0.07285189628601074, False),
  ('Ta', 0.08547395467758179, False),
  ('Mn', 0.08579063415527344, False),
  ('Ti', 0.09940117597579956, True

Cada site `AnonymousMotif` pode ser acessado individualmente através do atributo `sites`. Isso fornece flexibilidade para obter recomendações para um site específico do Anonymous Motif (AM). Para fazer isso, basta inserir o rótulo do site no método `RecommenderSystem.get_recommendation_for_site`. Esta abordagem permite uma exploração mais direcionada de potenciais substituições iônicas.

In [15]:
kagome_AM.sites

[(191, 9, (1, '6/mmm'), 0)
 site index: 0,
 (191, 9, (1, '6/mmm'), 0)
 site index: 3,
 (191, 9, (3, 'mmm'), 0)
 site index: 6,
 (191, 9, (4, '3m'), 0)
 site index: 1]

In [16]:
C_site_label = kagome_AM.sites[2].label
# top_n defined to get the top 10 recommendations ranked by the ion-site distance
rs.get_recommendation_for_site(C_site_label, top_n=10) 

[('V', 0.03470265865325928, False),
 ('Nb', 0.07285189628601074, False),
 ('Ta', 0.08547395467758179, False),
 ('Mn', 0.08579063415527344, False),
 ('Ti', 0.09940117597579956, True),
 ('Cr', 0.10330319404602051, False),
 ('Mo', 0.1266164779663086, False),
 ('Re', 0.13882142305374146, True),
 ('Fe', 0.15163779258728027, False),
 ('Hf', 0.1933293342590332, True)]

In [17]:
# Only new ion-site occupations added to filter recommendations
rs.get_recommendation_for_site(C_site_label, top_n=10, only_new=True) 

[('Ti', 0.09940117597579956, True),
 ('Re', 0.13882142305374146, True),
 ('Hf', 0.1933293342590332, True)]

### 1.2 Atomic substitutions for a structure defined on a CIF file

Também pode haver situações em que não temos um ID OQMD para o protótipo estrutural que desejamos investigar, mas temos um arquivo CIF. Por exemplo, suponha que estejamos interessados em explorar possíveis substituições iônicas no [mineral Jakobssonita](https://www.mindat.org/min-42796.html), um composto com composição $\text{CaAlF}_5$.

<p alinhar="centro">
      <img src="etc/jakobssonite.png" alt="Estrutura cristalina de CaAlF5" width="65%" class="center"> 
</p>

Usando o arquivo CIF fornecido pela Mindat, podemos carregar a estrutura cristalina Jakobssonita como um objeto Pymatgen `Structure`. Essa estrutura pode então ser alimentada no objeto `OccupationData`, que por sua vez gera seu correspondente `AnonymousMotif`. Armados com este `AnonymousMotif`, podemos prosseguir para gerar recomendações de substituição iônica para cada site da estrutura.

In [18]:
jakobssonite_structure = Structure.from_file('etc/jakobssonite.cif')
jakobssonite_structure

Structure Summary
Lattice
    abc : 8.712 6.317 7.349
 angles : 90.0 115.04000000000003 90.0
 volume : 366.4301426035834
      A : 7.89318110031551 0.0 -3.6873616743712625
      B : 1.0158510778007315e-15 6.317 3.868046915106915e-16
      C : 0.0 0.0 7.349
    pbc : True True True
PeriodicSite: Ca (5.51e-16, 3.426, 1.837) [0.0, 0.5424, 0.25]
PeriodicSite: Ca (3.947, 0.2678, -0.006431) [0.5, 0.0424, 0.25]
PeriodicSite: Ca (4.649e-16, 2.891, 5.512) [0.0, 0.4576, 0.75]
PeriodicSite: Ca (3.947, 6.049, 3.668) [0.5, 0.9576, 0.75]
PeriodicSite: Al (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Al (3.947, 3.159, -1.844) [0.5, 0.5, 0.0]
PeriodicSite: Al (0.0, 0.0, 3.675) [0.0, 0.0, 0.5]
PeriodicSite: Al (3.947, 3.159, 1.831) [0.5, 0.5, 0.5]
PeriodicSite: F1 (F) (9.571e-16, 5.952, 1.837) [0.0, 0.9422, 0.25]
PeriodicSite: F1 (F) (3.947, 2.793, -0.006431) [0.5, 0.4422, 0.25]
PeriodicSite: F1 (F) (5.872e-17, 0.3651, 5.512) [0.0, 0.0578, 0.75]
PeriodicSite: F1 (F) (3.947, 3.524, 3.668) [0.5, 0.5578, 

In [19]:
jakobssonite_AM = occupation_data.get_AM_from_structure(jakobssonite_structure)
jakobssonite_AM

(15, 14, [(2, '-1'), (2, '2'), (2, '2'), (4, '1'), (4, '1')], 0)

In [20]:
jakobssonite_recommendations = rs.get_recommendation_for_AM(jakobssonite_AM)
jakobssonite_recommendations

{'15_14_2(-1)_2(2)_2(2)_4(1)_4(1)_0[0:2(-1)]': [('Na',
   0.03471493721008301,
   False),
  ('K', 0.04308205842971802, False),
  ('Rb', 0.07844400405883789, False),
  ('Cs', 0.11494922637939453, True),
  ('Tl', 0.14745205640792847, False),
  ('Li', 0.21496635675430298, False),
  ('Ba', 0.30862218141555786, True)],
 '15_14_2(-1)_2(2)_2(2)_4(1)_4(1)_0[1:2(2)]': [('B',
   0.0044931769371032715,
   False),
  ('C', 0.16621392965316772, True),
  ('Al', 0.3119314908981323, False),
  ('Pt', 0.317658007144928, True),
  ('Si', 0.3232836127281189, True),
  ('Ga', 0.3240598440170288, True),
  ('N', 0.32569217681884766, True)],
 '15_14_2(-1)_2(2)_2(2)_4(1)_4(1)_0[2:2(2)]': [('S',
   0.012279212474822998,
   False),
  ('Se', 0.022431671619415283, False),
  ('Te', 0.12491989135742188, False)],
 '15_14_2(-1)_2(2)_2(2)_4(1)_4(1)_0[3:4(1)]': [('K',
   0.05715596675872803,
   False),
  ('Na', 0.0644150972366333, False),
  ('Rb', 0.07851749658584595, False),
  ('Cs', 0.14788156747817993, True),
  ('Tl', 0

## 2. Recomendações ao longo da hiperdireção estrutural

### 2.1. Novas estruturas cristalinas para AlCd

Podemos extrair recomendações de estruturas cristalinas para um determinado composto químico. Isso é possível graças ao método `get_recommendation_for_compound()`. Vamos demonstrá-lo para o caso do composto químico binário AlCd. O [sistema binário Al-Cd] (https://www.oqmd.org/materials/composition/Al-Cd) não possui materiais estáveis ​​em DFT nem quaisquer compostos experimentalmente conhecidos/realizados.

In [21]:
AlCd_recs = rs.get_recommendation_for_compound(formula='AlCd', top_n= 700)

Abaixo imprimimos as recomendações com uma soma das distâncias átomos do AMSite menor que 0,4. Isso equivale a quatro recomendações, com grupos espaciais Pbcm (57), C2/m (12) e Pnma (62), nenhum deles presente na base de dados OQMD.

In [22]:
df = DataFrame(AlCd_recs)
df[ df.sum_SiteAtom_dists<0.4 ].sort_values(by=['sum_SiteAtom_dists'])

,formula,AM,AM_label,atom_AMsites,oxidation_states,sum_SiteAtom_dists,novelty_fraction
3,AlCd,"(57, 8, [(4, 'm'), (4, 'm')], 4)",57_8_4(m)_4(m)_4,"[Cd, Al]",[],0.268250,1.0
2,AlCd,"(12, 4, [(2, 'm'), (2, 'm')], 0)",12_4_2(m)_2(m)_0,"[Cd, Al]",[],0.280006,0.5
0,AlCd,"(62, 8, [(4, 'm'), (4, 'm')], 2)",62_8_4(m)_4(m)_2,"[Cd, Al]",[],0.325816,0.5
1,AlCd,"(62, 8, [(4, 'm'), (4, 'm')], 2)",62_8_4(m)_4(m)_2,"[Al, Cd]",[],0.342469,0.5


## 3. Recomendações ao longo de hiperdireções híbridas composicionais-estruturais

### 3.1. Recomendando sites para o átomo de vanádio

Podemos recomendar sites AM para um átomo específico ocupar usando o método `get_recommendation_for_ion()`.

In [23]:
rs.get_recommendation_for_ion('V', top_n=200, only_new=True)

[((187, 6, (1, '-6m2'), 5)
  site index: 0,
  0.038537442684173584,
  True),
 ((187, 6, (1, '-6m2'), 5)
  site index: 0,
  0.04904705286026001,
  True),
 ((129, 8, (2, '4mm'), 5)
  site index: 2,
  0.06081873178482056,
  True),
 ((1, 23, (1, '1'), 7)
  site index: 21,
  0.06563454866409302,
  True),
 ((1, 23, (1, '1'), 7)
  site index: 21,
  0.06583338975906372,
  True),
 ((57, 16, (4, 'm'), 0)
  site index: 12,
  0.06810534000396729,
  True),
 ((63, 8, (2, 'm2m'), 3)
  site index: 6,
  0.07121402025222778,
  True),
 ((2, 12, (2, '1'), 16)
  site index: 2,
  0.07439231872558594,
  True),
 ((2, 20, (1, '-1'), 0)
  site index: 19,
  0.07455205917358398,
  True),
 ((12, 18, (2, 'm'), 1)
  site index: 2,
  0.07464230060577393,
  True),
 ((12, 18, (2, 'm'), 1)
  site index: 2,
  0.07631582021713257,
  True),
 ((12, 18, (1, '2/m'), 1)
  site index: 0,
  0.0766226053237915,
  True),
 ((12, 18, (1, '2/m'), 1)
  site index: 0,
  0.0766453742980957,
  True),
 ((2, 20, (1, '-1'), 0)
  site index:

### 3.2. Recomendação de locais para cobre que possuem geometria local coplanar quadrada

Podemos restringir as recomendações de sítios AM a íons, especificando a geometria local dos sítios a serem recomendados. Os nomes dos ambientes de geometrias locais são definidos de acordo com as dimensões da impressão digital do CrystalNN. Eles podem ser acessados através do atributo `OccupationData` `cnn_labels_dict`.

In [24]:
list(occupation_data.cnn_labels_dict.keys())

['wt CN_1',
 'sgl_bd CN_1',
 'wt CN_2',
 'L-shaped CN_2',
 'water-like CN_2',
 'bent 120 degrees CN_2',
 'bent 150 degrees CN_2',
 'linear CN_2',
 'wt CN_3',
 'trigonal planar CN_3',
 'trigonal non-coplanar CN_3',
 'T-shaped CN_3',
 'wt CN_4',
 'square co-planar CN_4',
 'tetrahedral CN_4',
 'rectangular see-saw-like CN_4',
 'see-saw-like CN_4',
 'trigonal pyramidal CN_4',
 'wt CN_5',
 'pentagonal planar CN_5',
 'square pyramidal CN_5',
 'trigonal bipyramidal CN_5',
 'wt CN_6',
 'hexagonal planar CN_6',
 'octahedral CN_6',
 'pentagonal pyramidal CN_6',
 'wt CN_7',
 'hexagonal pyramidal CN_7',
 'pentagonal bipyramidal CN_7',
 'wt CN_8',
 'body-centered cubic CN_8',
 'hexagonal bipyramidal CN_8',
 'wt CN_9',
 'q2 CN_9',
 'q4 CN_9',
 'q6 CN_9',
 'wt CN_10',
 'q2 CN_10',
 'q4 CN_10',
 'q6 CN_10',
 'wt CN_11',
 'q2 CN_11',
 'q4 CN_11',
 'q6 CN_11',
 'wt CN_12',
 'cuboctahedral CN_12',
 'q2 CN_12',
 'q4 CN_12',
 'q6 CN_12',
 'wt CN_13',
 'wt CN_14',
 'wt CN_15',
 'wt CN_16',
 'wt CN_17',
 'wt

Digamos que queremos novos sítios AM recomendados para Cu, mas com geometria local co-planar quadrada. Definimos uma pontuação mínima de 0,8 para a geometria dada.

In [25]:
Cu_recs = rs.get_recommendation_for_ion('Cu', local_geometry=('square co-planar CN_4', 0.8), only_new=True)
Cu_recs

[((136, 14, (2, 'mmm'), 0)
  site index: 8,
  0.14990442991256714,
  True),
 ((12, 12, (1, '2/m'), 7)
  site index: 11,
  0.1651182770729065,
  True),
 ((12, 12, (1, '2/m'), 7)
  site index: 11,
  0.1692206859588623,
  True),
 ((127, 18, (2, 'mmm'), 1)
  site index: 16,
  0.17258787155151367,
  True)]

Para obter a estrutura do primeiro AMSite recomendado (índice de site 8) e dos sites equivalentes

In [26]:
Cu_AM_label = Cu_recs[0][0].AM_label
Cu_AM = occupation_data.get_AnonymousMotif(Cu_AM_label)
Cu_AM_example_structure = Cu_AM.example_structure
Cu_AM_equivalent_sites_index = Cu_AM.equivalent_sites_indexes
Cu_AM_equivalent_sites_index

[0, 0, 2, 2, 0, 0, 2, 2, 8, 8, 10, 10, 10, 10]

In [27]:
Cu_AM_example_structure

Structure Summary
Lattice
    abc : 5.826722 5.826722 8.265963
 angles : 90.0 90.0 90.0
 volume : 280.6351412913347
      A : 5.826722 0.0 0.0
      B : 0.0 5.826722 0.0
      C : 0.0 0.0 8.265963
    pbc : True True True
PeriodicSite: C (1.178, 1.178, 0.0) [0.2021, 0.2021, 0.0]
PeriodicSite: C (4.649, 4.649, 0.0) [0.7979, 0.7979, 0.0]
PeriodicSite: C (0.0, 0.0, 1.666) [0.0, 0.0, 0.2015]
PeriodicSite: C (2.913, 2.913, 2.467) [0.5, 0.5, 0.2985]
PeriodicSite: C (4.091, 1.736, 4.133) [0.7021, 0.2979, 0.5]
PeriodicSite: C (1.736, 4.091, 4.133) [0.2979, 0.7021, 0.5]
PeriodicSite: C (2.913, 2.913, 5.799) [0.5, 0.5, 0.7015]
PeriodicSite: C (0.0, 0.0, 6.6) [0.0, 0.0, 0.7985]
PeriodicSite: C (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: C (2.913, 2.913, 4.133) [0.5, 0.5, 0.5]
PeriodicSite: C (2.913, 0.0, 2.066) [0.5, 0.0, 0.25]
PeriodicSite: C (0.0, 2.913, 2.066) [0.0, 0.5, 0.25]
PeriodicSite: C (2.913, 0.0, 6.199) [0.5, 0.0, 0.75]
PeriodicSite: C (0.0, 2.913, 6.199) [0.0, 0.5, 0.75]

Também podemos ter acesso aos ids do OQMD que apresentam este AM.

In [28]:
Cu_AM.entries_ids

[12415,
 12416,
 12881,
 12882,
 22733,
 1347628,
 1347810,
 1348154,
 1363514,
 1382773,
 1752433,
 1752436]

Acessando a [primeira entrada com id 12415](https://oqmd.org/materials/entry/12415), podemos visualizar um exemplo da estrutura cristalina do AM e os sítios de geometria co-planar que foram recomendados para o íon Cu ocupar .

<img src="etc/Rb2H4Pt.png" alt="Estrutura cristalina de Rb2H4Pt" class="center" width="65%" height="auto">

### 3.3. Recomendando novos compostos no sistema binário In-Co

Além disso, com o método `get_recommendation_for_chemsys()` também podemos explorar sistemas químicos definidos pelo usuário. Vamos demonstrar isso com o [sistema binário In-Co](https://www.oqmd.org/materials/composition/Co-In). Este sistema possui 2 fases experimentais conhecidas: In<sub>3</sub>Co (P2/mnm) e In<sub>2</sub>Co (Fddd).

In [29]:
InCo_recs = rs.get_recommendation_for_chemsys(elements=['In','Co'], top_n= 700)

Abaixo imprimimos as recomendações com uma soma das distâncias átomos do AMSite menor que 0,4. Isto resulta em 7 recomendações, duas das quais correspondem a estequiometrias não presentes no OQMD: In<sub>9</sub>Co<sub>2</sub> com grupo espacial P2/m (10), e In<sub> 10</sub>Co com grupo espacial P4/mmm (123). Curiosamente, essas duas novas estequiometrias têm uma pequena "soma das distâncias AMSite-átomo", o que sugere que podem ser estáveis ​​em DFT.

In [30]:
df = DataFrame(InCo_recs)
df[ df.sum_SiteAtom_dists<0.4 ].sort_values(by=['sum_SiteAtom_dists'])

,formula,AM,AM_label,atom_AMsites,oxidation_states,sum_SiteAtom_dists,novelty_fraction
8,In9Co2,"(10, 11, [(1, '2/m'), (2, 'm'), (2, 'm'), (2, ...",10_11_1(2/m)_2(m)_2(m)_2(m)_2(m)_2(m)_0,"[In, Co, In, In, In, In]",[],0.103411,0.0
3,In3Co,"(136, 16, [(4, '2/m'), (4, 'm2m'), (8, 'm')], 0)",136_16_4(2/m)_4(m2m)_8(m)_0,"[In, Co, In]",[],0.113033,0.0
10,In10Co,"(123, 11, [(1, '4/mmm'), (2, 'mmm'), (2, '4mm'...",123_11_1(4/mmm)_2(mmm)_2(4mm)_2(4mm)_4(2mm)_0,"[Co, In, In, In, In]",[],0.133173,0.0
0,InCo,"(57, 8, [(4, 'm'), (4, 'm')], 2)",57_8_4(m)_4(m)_2,"[Co, In]",[],0.223490,1.0
9,In3Co2,"(164, 5, [(1, '-3m'), (2, '3m'), (2, '3m')], 3)",164_5_1(-3m)_2(3m)_2(3m)_3,"[In, Co, In]",[],0.232919,0.0
6,In4Co,"(229, 5, [(1, 'm-3m'), (4, '-3m')], 0)",229_5_1(m-3m)_4(-3m)_0,"[Co, In]",[],0.291753,1.0
1,In2Co,"(123, 3, [(1, '4/mmm'), (2, '4mm')], 0)",123_3_1(4/mmm)_2(4mm)_0,"[Co, In]",[],0.299549,1.0
